In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from datetime import datetime
import pandas as pd 
import numpy as np 
import keras
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.model_selection import train_test_split
from keras.models import Sequential 
from keras.layers import Dense, Flatten, Dropout, LSTM, GRU
from sklearn.metrics import mean_squared_error
from keras.layers import Dropout,BatchNormalization

os.chdir(r'D:\Github 작업용\Github_cowork_practice')

In [ ]:
train_data = pd.read_csv('Embedding_train_data.csv')
test_data = pd.read_csv('Embedding_test_data.csv')

In [ ]:
#정규화
#data = df.drop(['ymdhm', 'ymd'],axis=1)

data = df.loc[:,['swl', 'inf', 'sfw', 'ecpc', 'tototf', 'tide_level',
                 'rf_10184100', 'rf_10184110', 'rf_10184140',
                 'wl_1018662', 'fw_1018662', 'wl_1018680', 'wl_1018683',
                 'fw_1018683', 'wl_1019630', 'fw_1019630']]

target = df.loc[:,['wl_1018662', 'wl_1018680', 'wl_1018683', 'wl_1019630']]

data_scaler = MinMaxScaler()
data_scaled = data_scaler.fit_transform(data)

target_scaler = MinMaxScaler()
target_scaled = target_scaler.fit_transform(target)

#전처리
window_size = 144 # 10분 단위

data_list = []
target_list = []

for i in range(window_size, len(data_scaled)):
    data_list.append(np.array(data_scaled[i-window_size:i]))
    target_list.append(np.array(target_scaled[i]))

data_ar = np.array(data_list)
target_ar = np.array(target_list)

print(np.shape(data_ar), np.shape(target_ar))

In [16]:
# train set, test set 분할 
x_train, x_test, y_train, y_test = train_test_split(data_ar, target_ar, test_size=0.2, shuffle=False, random_state = 2021)

In [17]:
model = Sequential()
model.add(LSTM(128, input_shape=(window_size, 16), return_sequences = True))
model.add(LSTM(128, return_sequences = True))
model.add(LSTM(128))
model.add(Dense(128, activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(128,activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(64,activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(4))

# model compile
model.compile(loss='mse', optimizer='adam') 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 144, 128)          74240     
                                                                 
 lstm_1 (LSTM)               (None, 144, 128)          131584    
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0

In [ ]:
# model setting
early_stopping = keras.callbacks.EarlyStopping(patience = 15, restore_best_weights = True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
history = model.fit(x_train, y_train, batch_size=256, epochs=500, callbacks=[early_stopping, reduce_lr], validation_data=(x_test, y_test))

Epoch 1/500
842/842 [==============================] - 46s 48ms/step - loss: 3.5455e-04 - val_loss: 1.4118e-04 - lr: 0.0010
Epoch 2/500
842/842 [==============================] - 38s 45ms/step - loss: 1.5616e-04 - val_loss: 1.1104e-04 - lr: 0.0010
Epoch 3/500
842/842 [==============================] - 38s 45ms/step - loss: 1.3280e-04 - val_loss: 1.2217e-04 - lr: 0.0010
Epoch 4/500
842/842 [==============================] - 40s 47ms/step - loss: 1.1989e-04 - val_loss: 1.2549e-04 - lr: 0.0010
Epoch 5/500
842/842 [==============================] - 37s 44ms/step - loss: 1.1491e-04 - val_loss: 2.5552e-04 - lr: 0.0010
Epoch 6/500
842/842 [==============================] - 43s 51ms/step - loss: 1.0758e-04 - val_loss: 8.1389e-05 - lr: 0.0010
Epoch 7/500
842/842 [==============================] - 38s 45ms/step - loss: 9.8328e-05 - val_loss: 1.0158e-04 - lr: 5.0000e-04
Epoch 8/500
842/842 [==============================] - 40s 47ms/step - loss: 9.8948e-05 - val_loss: 1.2349e-04 - lr: 5.0000e-04


In [ ]:
#model.save(str(window_size)+'model_lstm-16col_11+sgfilter15-2.h5')
model = keras.models.load_model('144model_3lstm-4dense.h5')

In [ ]:
#train 예측 
train_predictions = model.predict(x_train)

#train 역변환 
val = target_scaler.inverse_transform(y_train)
predictions = target_scaler.inverse_transform(train_predictions)

#train RMSE
train_RMSE = mean_squared_error(val, predictions)**0.5

#test 예측 
test_predictions = model.predict(x_test)

#test 역변환 
val = target_scaler.inverse_transform(y_test)
predictions = target_scaler.inverse_transform(test_predictions)

#test RMSE
test_RMSE = mean_squared_error(val, predictions)**0.5

print('train_RMSE:', train_RMSE)
print('test_RMSE:', test_RMSE)
